# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 1:15PM,240970,15,6658991,"Person & Covey, Inc.",Executing
1,Jul 1 2022 1:15PM,240969,18,INNOVA-8218254,Innova Softgel LLC,Executing
2,Jul 1 2022 12:59PM,240967,10,8218247,Eywa Pharma Inc.,Released
3,Jul 1 2022 12:50PM,240966,19,GP07052022,"Granules Pharmaceuticals, Inc.",Released
4,Jul 1 2022 12:45PM,240964,19,8217675,"Innogenix, LLC",Executing
5,Jul 1 2022 12:45PM,240964,19,8217676,"Innogenix, LLC",Released
6,Jul 1 2022 12:45PM,240964,19,8217677,"Innogenix, LLC",Released
7,Jul 1 2022 12:45PM,240964,19,8217678,"Innogenix, LLC",Executing
8,Jul 1 2022 12:45PM,240964,19,8217679,"Innogenix, LLC",Executing
9,Jul 1 2022 12:45PM,240964,19,8217680,"Innogenix, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,240964,Released,5
24,240966,Released,1
25,240967,Released,1
26,240969,Executing,1
27,240970,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240964,NaN,15.0,5.0
240966,NaN,NaN,1.0
240967,NaN,NaN,1.0
240969,NaN,1.0,NaN
240970,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240873,0.0,0.0,1.0
240878,0.0,0.0,1.0
240892,0.0,1.0,0.0
240893,0.0,0.0,1.0
240896,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240873,0,0,1
240878,0,0,1
240892,0,1,0
240893,0,0,1
240896,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240873,0,0,1
1,240878,0,0,1
2,240892,0,1,0
3,240893,0,0,1
4,240896,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240873,,,1
1,240878,,,1
2,240892,,1,
3,240893,,,1
4,240896,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc."
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc."
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC"
24,Jul 1 2022 12:34PM,240960,10,Bio-PRF
40,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc."
59,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc."
70,Jul 1 2022 12:08PM,240958,10,Beach Products Inc
71,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc.",,1,
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC,,1,
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,,,1
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc.",,,1
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC",,15,5
5,Jul 1 2022 12:34PM,240960,10,Bio-PRF,,,16
6,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc.",,,19
7,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",,,11
8,Jul 1 2022 12:08PM,240958,10,Beach Products Inc,,,1
9,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,,,25


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc.",,1,
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC,,1,
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,1,,
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc.",1,,
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC",5,15,
5,Jul 1 2022 12:34PM,240960,10,Bio-PRF,16,,
6,Jul 1 2022 12:31PM,240962,20,"Exact-Rx, Inc.",19,,
7,Jul 1 2022 12:23PM,240959,20,"ACI Healthcare USA, Inc.",11,,
8,Jul 1 2022 12:08PM,240958,10,Beach Products Inc,1,,
9,Jul 1 2022 12:06PM,240956,20,Alumier Labs Inc.,25,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc.",,1,
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC,,1,
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,1,,
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc.",1,,
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC",5,15,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc.",NaN,1.0,NaN
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC,NaN,1.0,NaN
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC",5.0,15.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 1 2022 1:15PM,240970,15,"Person & Covey, Inc.",0.0,1.0,0.0
1,Jul 1 2022 1:15PM,240969,18,Innova Softgel LLC,0.0,1.0,0.0
2,Jul 1 2022 12:59PM,240967,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,Jul 1 2022 12:50PM,240966,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0
4,Jul 1 2022 12:45PM,240964,19,"Innogenix, LLC",5.0,15.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2168470,88.0,30.0,4.0
15,240970,0.0,1.0,0.0
16,963601,13.0,1.0,0.0
18,240969,0.0,1.0,0.0
19,963719,8.0,15.0,0.0
20,963832,56.0,3.0,0.0
21,240949,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2168470,88.0,30.0,4.0
1,15,240970,0.0,1.0,0.0
2,16,963601,13.0,1.0,0.0
3,18,240969,0.0,1.0,0.0
4,19,963719,8.0,15.0,0.0
5,20,963832,56.0,3.0,0.0
6,21,240949,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,30.0,4.0
1,15,0.0,1.0,0.0
2,16,13.0,1.0,0.0
3,18,0.0,1.0,0.0
4,19,8.0,15.0,0.0
5,20,56.0,3.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,15,Released,0.0
2,16,Released,13.0
3,18,Released,0.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,30.0,1.0,1.0,1.0,15.0,3.0,0.0
Released,88.0,0.0,13.0,0.0,8.0,56.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,30.0,1.0,1.0,1.0,15.0,3.0,0.0
2,Released,88.0,0.0,13.0,0.0,8.0,56.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,30.0,1.0,1.0,1.0,15.0,3.0,0.0
2,Released,88.0,0.0,13.0,0.0,8.0,56.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()